In [1]:
import ee
from pathlib import Path
import pandas as pd
import ast

ee.Authenticate() #Uncomment this whenever needed, once done usually not needed for 1-2 days
ee.Initialize(project='raman-461708')

AEZ_no = 15
asset_list = ee.data.listAssets({'parent': 'projects/raman-461708/assets/'})['assets']

# Filter for boundaries assets that match the AEZ_no
boundary_assets = [
    asset['name'] for asset in asset_list
    if f'AEZ_{AEZ_no}_boundaries' in asset['name']
]

# Load and merge all matching boundaries for this AEZ_no
aez_boundaries = ee.FeatureCollection([])
for asset_path in boundary_assets:
    fc = ee.FeatureCollection(asset_path)
    aez_boundaries = aez_boundaries.merge(fc)

import pandas as pd

# Read the CSV file
df = pd.read_csv("AEZ_"+str(AEZ_no)+"/status.csv")

# Function to create a tile (rectangle polygon) from diagonal points
def create_tile_from_points(points):
    # points is a string representation of a tuple, e.g. "((lat1, lon1), (lat2, lon2))"
    if isinstance(points, str):
        points_tuple = ast.literal_eval(points)
    else:
        points_tuple = points
    (lon1, lat1), (lon2, lat2) = points_tuple
    # Get min/max for lat/lon to define the rectangle
    min_lat, max_lat = min(lat1, lat2), max(lat1, lat2)
    min_lon, max_lon = min(lon1, lon2), max(lon1, lon2)
    # Return the four corners in order (clockwise or counterclockwise)
    return [
        (min_lat, min_lon),
        (min_lat, max_lon),
        (max_lat, max_lon),
        (max_lat, min_lon),
        (min_lat, min_lon)  # close the polygon
    ]

# Create a new column 'tile' with the rectangle polygon for each row
df['tile'] = df['points'].apply(create_tile_from_points)
tiles = df['tile'].tolist()

samples = []



def fc_to_df(fc):
    # Get the data from the FeatureCollection
    features = fc.getInfo()['features']
    
    # Extract properties and geometry
    data = [feature['properties'] for feature in features]
    
    return pd.DataFrame(data)

def process_tile(tile):
    tile = ee.Geometry.Polygon(tile).buffer(-5)
    
    mapping = {
        "farm": 1,
        "plantation": 2,
        "scrubland": 3,
        "rest": 0
    }
    reversed_mapping = {v: k for k, v in mapping.items()}
    reversed_ee_mapping = ee.Dictionary(reversed_mapping)

    easy_farm = [
        ee.Filter.gte("rect", 0.67),
        ee.Filter.gt("size", 500),
        ee.Filter.lt("size", 2000),
        ee.Filter.lt("ent", 1)
        ]
    easy_scrubland = [
        ee.Filter.gte("size", 60000),
        ee.Filter.lte("size", 5000000)
        ]
    easy_plantation = [
        ee.Filter.lt("area", 20000),
        ee.Filter.gt("area", 1000)
    ]

    all = aez_boundaries.filterBounds(tile)
    farm = all.filter(ee.Filter.And(*easy_farm))
    scrubland = all.filter(ee.Filter.And(easy_scrubland))
    plantation = all.filter(ee.Filter.eq("class", "plantation")).map(lambda x: x.set("area", x.geometry().area())).filter(ee.Filter.And(easy_plantation))
    mapping = {
        "farm": 1,
        "plantation": 2,
        "scrubland": 3,
        "rest": 0
    }
    reversed_mapping = {v: k for k, v in mapping.items()}
    reversed_ee_mapping = ee.Dictionary(reversed_mapping)

    easy_farm = [
        ee.Filter.gte("rect", 0.67),
        ee.Filter.gt("size", 500),
        ee.Filter.lt("size", 2000),
        ee.Filter.lt("ent", 1)
        ]

    easy_scrubland = [
        ee.Filter.gte("size", 60000)
        ]

    
    
    #Filter out farms which doesnot have 3 nearby farms (Removing solo farms inside scrublands)
    farm_buffer = farm.map(lambda x: x.buffer(10))
    # Merge touching farms into clusters
    farm_buffer_union = farm_buffer.union(maxError=1)
    geoms = farm_buffer_union.geometry().dissolve().geometries()
    geoms = geoms.map(lambda geom: ee.Feature(ee.Geometry(geom)))
    # Flatten clusters into individual polygons
    clusters = ee.FeatureCollection(geoms)
    clusters = clusters.map(lambda x: x.set("count", farm.filterBounds(x.geometry()).size())).filter(ee.Filter.gte('count', 3))
    farm = farm.filterBounds(clusters)
    
    #Filtering out scrublands
    lulc_v3 = ee.Image("projects/corestack-datasets/assets/datasets/LULC_v3_river_basin/pan_india_lulc_v3_2023_2024").clip(tile)
    classes_of_interest = [8, 9, 10, 11]
    
    
    # Build a binary mask for classes of interest (1 = of interest, 0 = otherwise)
    interest_mask = (
        lulc_v3
        .remap(classes_of_interest, ee.List.repeat(1, len(classes_of_interest)), 0)
        .rename('interestMask')
        .unmask(0)
    )

    # Pixel area bands
    px_area       = ee.Image.pixelArea().rename('total_area')
    interest_area = px_area.updateMask(interest_mask).rename('interest_area')

    # Stack and clip to the tile
    area_stack = interest_area.addBands(px_area).clip(tile)

    # Sum both bands over all polygons in one call
    summed = area_stack.reduceRegions(
    collection=scrubland,
    reducer=ee.Reducer.sum(),
    scale=10
    )

    # Compute percentage per feature (no if-condition needed)
    with_percent = summed.map(
        lambda f: f.set(
            'percent_interest',
            ee.Number(f.get('interest_area'))
            .divide(ee.Number(f.get('total_area')))
            .multiply(100)
        )
    )

    # Filter: < 50% interest AND strictly inside the ROI
    scrubland = (
        with_percent
        .filter(ee.Filter.lt('percent_interest', 50))
    )
    # Desired samples per class
    class_values = [(0,0),(1, 150), (2, 150), (3, 150)]
    class_band = 'label'

    # Build the label raster (priority = last paint wins for overlaps).
    # If you need explicit precedence, keep this order intentional.
    label_image = (
        ee.Image(0).byte().rename(class_band)
        .paint(farm,       mapping["farm"])
        .paint(scrubland,  mapping["scrubland"])
        .paint(plantation, mapping["plantation"])
        .clip(tile)
    )

    # Prepare lists for stratified sampling
    class_values_list  = [v for v, _ in class_values]
    class_points_list  = [n for _, n in class_values]

    # One stratified sample call; returns up to the requested points per class
    all_samples = label_image.stratifiedSample(
        numPoints=sum(class_points_list),
        classBand=class_band,
        classValues=class_values_list,
        classPoints=class_points_list,
        scale=10,                 # your LULC pixel size
        region=tile,
        seed=42,
        dropNulls=True,
        geometries=True
    )
    return farm, scrubland, plantation


farms, scrublands, plantations = process_tile(tiles[1])

FileNotFoundError: [Errno 2] No such file or directory: 'AEZ_15/status.csv'

In [2]:
import geemap
import ee
import pandas as pd
import ast

# Set AEZ number here
AEZ_no = 15

# Initialize Earth Engine
# ee.Authenticate()
# ee.Initialize()

# Read the CSV
df = pd.read_csv(f"AEZ_{AEZ_no}/status.csv")

# Function to create a rectangle polygon from the 'points' string
def create_polygon(points):
    (lat1, lon1), (lat2, lon2) = ast.literal_eval(points)
    min_lat, max_lat = min(lat1, lat2), max(lat1, lat2)
    min_lon, max_lon = min(lon1, lon2), max(lon1, lon2)
    coords = [
        [min_lon, min_lat],
        [min_lon, max_lat],
        [max_lon, max_lat],
        [max_lon, min_lat],
        [min_lon, min_lat]
    ]
    return ee.Geometry.Polygon([coords])

# Build list of ee.Feature
features = []
for _, row in df.iterrows():
    geom = create_polygon(row['points'])
    props = row.drop('points').to_dict()
    features.append(ee.Feature(geom, props))

# Create the FeatureCollection
boundaries_boxes = ee.FeatureCollection(features)


# Example for boundaries if needed:
# boundaries_0 = ee.FeatureCollection(f"projects/raman-461708/assets/AEZ_{AEZ_no}_boundaries_0")
# boundaries_1 = ee.FeatureCollection(f"projects/raman-461708/assets/AEZ_{AEZ_no}_boundaries_1")
# boundaries = boundaries_0.merge(boundaries_1)

aez = ee.FeatureCollection("users/mtpictd/agro_eco_regions").filter(ee.Filter.eq("ae_regcode", AEZ_no))
tiles = ee.FeatureCollection(f"projects/raman-461708/assets/AEZ_{AEZ_no}_tiles")

# Example for centering the map (if you have tiles)
# bounds = tiles.geometry().bounds()
# coords = bounds.centroid(10).coordinates().getInfo()
# center = [coords[1], coords[0]]

# If you don't have tiles, use a default center
center = [21.54, 74.51]  # Replace with your region of interest

# Create a geemap interactive map
Map = geemap.Map(center=center, zoom=7, basemap='SATELLITE')
Map.addLayer(tiles, {}, "Tiles")

Map.addLayer(boundaries_boxes, {}, "Boundaries")
Map.addLayer(aez, {}, "AEZ")

Map

Map(center=[21.54, 74.51], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright'…

In [3]:
tiles.size()

In [17]:
def split_roi(fc):
    roi = ee.Feature(fc.first()).geometry().dissolve()

    # 3) Compute axis-aligned bounding box and its extents.
    bbox = roi.bounds(1)
    ring = ee.List(ee.List(bbox.coordinates()).get(0))
    xs = ring.map(lambda p: ee.Number(ee.List(p).get(0)))
    ys = ring.map(lambda p: ee.Number(ee.List(p).get(1)))
    minX = ee.Number(xs.reduce(ee.Reducer.min()))
    maxX = ee.Number(xs.reduce(ee.Reducer.max()))
    minY = ee.Number(ys.reduce(ee.Reducer.min()))
    maxY = ee.Number(ys.reduce(ee.Reducer.max()))

    width  = maxX.subtract(minX)
    height = maxY.subtract(minY)
    splitVertically = width.gte(height)  # If wider than tall → vertical split; else horizontal.

    xMid = minX.add(maxX).divide(2)
    yMid = minY.add(maxY).divide(2)

    # 4) Build two half-rectangles that cover the bbox.
    leftRect   = ee.Geometry.Rectangle([minX, minY, xMid,  maxY], proj=None, geodesic=False)
    rightRect  = ee.Geometry.Rectangle([xMid,  minY, maxX, maxY], proj=None, geodesic=False)
    bottomRect = ee.Geometry.Rectangle([minX, minY, maxX, yMid ], proj=None, geodesic=False)
    topRect    = ee.Geometry.Rectangle([minX, yMid,  maxX, maxY], proj=None, geodesic=False)

    # 5) Intersect ROI with the chosen halves to get two parts.
    part1 = ee.Geometry(ee.Algorithms.If(
        splitVertically, roi.intersection(leftRect, 1), roi.intersection(bottomRect, 1)
    ))
    part2 = ee.Geometry(ee.Algorithms.If(
        splitVertically, roi.intersection(rightRect, 1), roi.intersection(topRect, 1)
    ))

    # 6) Wrap as a FeatureCollection (two parts).
    parts_fc = ee.FeatureCollection([
        ee.Feature(part1).set({'part': 1}),
        ee.Feature(part2).set({'part': 2})
    ])

    # (Optional) sanity check: areas (m^2)
    print('Areas (m^2):', ee.Dictionary({
        'total': roi.area(1),
        'part1': part1.area(1),
        'part2': part2.area(1)
    }).getInfo())
    return parts_fc
    

In [33]:
# pip install geemap earthengine-api
import geemap


# --- Inputs ---
# 'aez' should be an ee.FeatureCollection with a numeric/string property 'ae_regcode'
# Example (replace with your asset if needed):
# aez = ee.FeatureCollection('users/your_username/aez')

label_prop = 'ae_regcode'  # change if your field name differs

aez = ee.FeatureCollection("users/mtpictd/agro_eco_regions")
# pip install geemap earthengine-api


# --- Inputs ---
# 'aez' should be an ee.FeatureCollection with a property 'ae_regcode'
# Example:
# aez = ee.FeatureCollection('users/your_username/aez')

label_prop = 'ae_regcode'  # change if your label field differs

# Style boundaries (transparent fill)
styled = aez.style(
    color='#1f77b4',
    width=2,
    fillColor='00000000'
)

def interior_label_point(f):
    geom = f.geometry()
    # 1) Try centroid
    centroid = ee.Geometry(geom.centroid(maxError=1))
    # 2) If centroid not inside, fall back to one random point inside polygon
    pt = ee.Algorithms.If(
        geom.contains(centroid, 1),
        centroid,
        ee.FeatureCollection.randomPoints(region=geom, points=1, maxError=1).first().geometry()
    )
    # Build label text (works for numeric or string)
    label = ee.String(f.get(label_prop))
    return ee.Feature(ee.Geometry(pt), {'label': label})

label_fc = aez.map(interior_label_point)

# Map
m = geemap.Map()
m.addLayer(styled, {}, 'AEZ boundaries')

# Add labels (requires geemap >= 0.27.0)
m.add_labels(
    label_fc,
    'label',                 # column (positional)
    font_size='14pt',        # <-- must include 'pt'
    font_color='black',
    outline_color='white',
    outline_width='3pt',     # safer to give 'pt' here too
    name='AEZ labels'
)
aez_6 = aez.filter(ee.Filter.eq("ae_regcode", 2))
aez_6_parts = split_roi(aez_6)
m.addLayer(ee.FeatureCollection(aez_6_parts), {'color': 'red'}, 'AEZ 6 - part 1')
samples = ee.FeatureCollection("projects/raman-461708/assets/gee_samples_all").filter(ee.Filter.eq("aez_no", 12))
m.addLayer(samples, {}, "Samples")
#m.addLayer(ee.FeatureCollection(aez_6_2), {'color': 'blue'}, 'AEZ 6 - part 2')
m.centerObject(aez, 5)
m


Areas (m^2): {'part1': 140736016247.09848, 'part2': 184869326886.93112, 'total': 325605343134.6092}


Map(center=[23.22104337560591, 79.46098827567988], controls=(WidgetControl(options=['position', 'transparent_b…

In [34]:
samples.size().getInfo()

120750

In [5]:
import geemap
import ee
import pandas as pd
import ast

ee.Initialize(project='raman-461708')

all_tiles = ee.FeatureCollection([])
all_boundaries = ee.FeatureCollection([])
all_aez = ee.FeatureCollection([])

for AEZ_no in range(1, 21):
    # Try to read the CSV for this AEZ (skip if not found)
    try:
        df = pd.read_csv(f"AEZ_{AEZ_no}/status.csv")
    except FileNotFoundError:
        print(f"AEZ_{AEZ_no}/status.csv not found, skipping.")
        continue

    # Build boundaries_boxes from CSV
    def create_polygon(points):
        (lat1, lon1), (lat2, lon2) = ast.literal_eval(points)
        min_lat, max_lat = min(lat1, lat2), max(lat1, lat2)
        min_lon, max_lon = min(lon1, lon2), max(lon1, lon2)
        coords = [
            [min_lon, min_lat],
            [min_lon, max_lat],
            [max_lon, max_lat],
            [max_lon, min_lat],
            [min_lon, min_lat]
        ]
        return ee.Geometry.Polygon([coords])

    features = []
    for _, row in df.iterrows():
        geom = create_polygon(row['points'])
        props = row.drop('points').to_dict()
        features.append(ee.Feature(geom, props))
    boundaries_boxes = ee.FeatureCollection(features)
    all_boundaries = all_boundaries.merge(boundaries_boxes)

    # AEZ boundary and tiles
    aez = ee.FeatureCollection("users/mtpictd/agro_eco_regions").filter(ee.Filter.eq("ae_regcode", AEZ_no))
    all_aez = all_aez.merge(aez)
    try:
        tiles = ee.FeatureCollection(f"projects/raman-461708/assets/AEZ_{AEZ_no}_tiles")
        all_tiles = all_tiles.merge(tiles)
    except Exception as e:
        print(f"Tiles for AEZ_{AEZ_no} not found or error: {e}")

Map = geemap.Map(center=[21.54, 74.51], zoom=5, basemap='SATELLITE')
Map.addLayer(all_tiles, {}, "All Tiles")
Map.addLayer(all_boundaries, {}, "All Boundaries")
Map.addLayer(all_aez, {}, "All AEZs")
Map

AEZ_20/status.csv not found, skipping.


Map(center=[21.54, 74.51], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright'…

In [3]:
import geemap

AEZ_no = 9
roi = ee.FeatureCollection("users/mtpictd/agro_eco_regions").filter(ee.Filter.eq("ae_regcode", AEZ_no)) 
original_lulc_img = ee.Image('projects/corestack-datasets/assets/datasets/LULC_v3_river_basin/pan_india_lulc_v3_2023_2024').clip(roi).select('predicted_label')
final_lulc_img = ee.Image('projects/raman-461708/assets/AEZ_'+str(AEZ_no)+'_v4')
m = geemap.Map()
#m.centerObject(farm.limit(1))


url = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}'
m.layout.height = '1000px'
m.add_tile_layer(url, name="Google Map", attribution="Google")
#m.add_basemap(, )
m.addLayerControl()
#m.addLayer(fields_boundary, {}, 'Fields')

palette = [
    '303000', '000000', '000000', '000000', '000000', 'f75cff', 
    '991695', 'e6ab2c', '3bef34', '2baa25', 'e6ef34', 'a1a524', 'eea15e'
]

palette_corrected = [
    '303000', '000000', '000000', '000000', '000000', 'f75cff', 
    '991695', 'e6ab2c', '3bef34', '2baa25', 'e6ef34', 'a1a524', 'eea15e', '29554E'
]

vis_params = {
    'min': 0,
    'max': 12,
    'palette': palette
}

vis_params_corrected = {
    'min': 0,
    'max': 13,
    'palette': palette_corrected
}

labels = """0. Background
1. Built-up
2. Water (Kharif)
3. Water (Kharif + Rabi)
4. Water (Kharif + Rabi + Zaid)
5. Croplands
6. Tree/Forest
7. Barren Land
8. Single Kharif (Light blue)
9. Single Non-Kharif
10. Double (Dark-Blue)
11. Triple
12. Shrub_Scrub
13. Plantation"""
labels = [i.split(". ")[-1] for i in labels.split("\n")]

palette_ = ['3bef34', '991695', '0000FF']
vis_params_ = {
    'min': 0,
    'max': 2,
    'palette': palette_
}

m.addLayer(final_lulc_img.select("predicted_label"), vis_params_corrected, 'LULC_corrected_4')
m.addLayer(original_lulc_img, vis_params, 'LULC_original')
m.addLayer(all_boundaries.filterBounds(roi), {}, "Sampling regions")
m.addLayer(farms, {}, "Farms")
m.addLayer(scrublands, {}, "Scrublands")
m.addLayer(plantations, {}, "Plantations")

m

NameError: name 'all_boundaries' is not defined

In [4]:
import geemap

AEZ_no = 3
roi = ee.FeatureCollection("users/mtpictd/agro_eco_regions").filter(ee.Filter.eq("ae_regcode", AEZ_no)) 
original_lulc_img = ee.Image('projects/corestack-datasets/assets/datasets/LULC_v3_river_basin/pan_india_lulc_v3_2023_2024').clip(roi).select('predicted_label')
final_lulc_img = ee.Image('projects/raman-461708/assets/AEZ_'+str(AEZ_no)+'_v4')
final_lulc_img_local = ee.Image('projects/raman-461708/assets/AEZ_'+str(AEZ_no)+'_v4_local')
m = geemap.Map()
#m.centerObject(farm.limit(1))


url = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}'
m.layout.height = '1000px'
m.add_tile_layer(url, name="Google Map", attribution="Google")
#m.add_basemap(, )
m.addLayerControl()
#m.addLayer(fields_boundary, {}, 'Fields')

palette = [
    '303000', '000000', '000000', '000000', '000000', 'f75cff', 
    '991695', 'e6ab2c', '3bef34', '2baa25', 'e6ef34', 'a1a524', 'eea15e'
]

palette_corrected = [
    '303000', '000000', '000000', '000000', '000000', 'f75cff', 
    '991695', 'e6ab2c', '3bef34', '2baa25', 'e6ef34', 'a1a524', 'eea15e', '29554E'
]

vis_params = {
    'min': 0,
    'max': 12,
    'palette': palette
}

vis_params_corrected = {
    'min': 0,
    'max': 13,
    'palette': palette_corrected
}

labels = """0. Background
1. Built-up
2. Water (Kharif)
3. Water (Kharif + Rabi)
4. Water (Kharif + Rabi + Zaid)
5. Croplands
6. Tree/Forest
7. Barren Land
8. Single Kharif (Light blue)
9. Single Non-Kharif
10. Double (Dark-Blue)
11. Triple
12. Shrub_Scrub
13. Plantation"""
labels = [i.split(". ")[-1] for i in labels.split("\n")]

palette_ = ['3bef34', '991695', '0000FF']
vis_params_ = {
    'min': 0,
    'max': 2,
    'palette': palette_
}

m.addLayer(final_lulc_img.select("predicted_label"), vis_params_corrected, 'LULC_corrected_4')
m.addLayer(final_lulc_img_local.select("predicted_label"), vis_params_corrected, 'LULC_corrected_4_local')
m.addLayer(original_lulc_img, vis_params, 'LULC_original')


m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

In [5]:
import re
original_lulc_img = ee.Image('projects/corestack-datasets/assets/datasets/LULC_v3_river_basin/pan_india_lulc_v3_2023_2024').select('predicted_label')
PARENT = 'projects/raman-461708/assets'

# ---- List all image assets once (fast follow-up lookups) ----
def _basename(asset_name: str) -> str:
    return asset_name.split('/')[-1]

all_assets = ee.data.listAssets({'parent': PARENT}).get('assets', [])
image_assets = [a for a in all_assets if a.get('type','').lower() == 'image']
basenames = {_basename(a['name']) for a in image_assets}

def asset_exists_base(base: str) -> bool:
    return base in basenames

def fullpath(base: str) -> str:
    return f'{PARENT}/{base}'

def list_part_bases_for_aez(aez_no: int):
    """Return sorted basenames for all parts like AEZ_{n}_v4_local_partK (K numeric)."""
    rx = re.compile(rf'^AEZ_{aez_no}_v4_local_part(\d+)$')
    parts = []
    for b in basenames:
        m = rx.match(b)
        if m:
            parts.append((int(m.group(1)), b))
    # Sort by numeric suffix to keep deterministic order (not required for mosaic, but nice to have).
    parts.sort(key=lambda x: x[0])
    return [b for _, b in parts]

def choose_assets_for_aez(aez_no: int):
    """
    Preference:
      1) AEZ_{n}_v4
      2) AEZ_{n}_v4_local_part*
      3) AEZ_{n}_v4_local
    Returns: list of full asset IDs (possibly multiple if parts), plus a metadata dict.
    """
    base_v4 = f'AEZ_{aez_no}_v4'
    base_local = f'AEZ_{aez_no}_v4_local'
    part_bases = list_part_bases_for_aez(aez_no)

    if asset_exists_base(base_v4):
        return [fullpath(base_v4)], {'source':'v4','parts':[]}

    if part_bases:
        return [fullpath(b) for b in part_bases], {'source':'v4_local_parts','parts':part_bases}

    if asset_exists_base(base_local):
        return [fullpath(base_local)], {'source':'v4_local','parts':[]}

    return [], {'source':'missing','parts':[]}

# ---- Collect images AEZ 1..19 ----
images = []
report = {}   # aez_no -> {'source':..., 'parts':[...] }
missing = []

for aez_no in range(1, 20):
    asset_ids, meta = choose_assets_for_aez(aez_no)
    report[aez_no] = meta
    if asset_ids:
        # If this AEZ has multiple parts, add them all; mosaic will merge them.
        for aid in asset_ids:
            images.append(ee.Image(aid))
    else:
        missing.append(aez_no)

if not images:
    raise RuntimeError("No AEZ images found across v4 / v4_local_parts / v4_local")

# ---- Final mosaic across all gathered images ----
# Order: AEZ 1..19; within an AEZ, parts in ascending numeric order.
# ImageCollection.mosaic() draws later images on top of earlier ones.
mosaic_img = ee.ImageCollection(images).mosaic()
m = geemap.Map()
#m.centerObject(farm.limit(1))


url = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}'
m.layout.height = '1000px'
m.add_tile_layer(url, name="Google Map", attribution="Google")
#m.add_basemap(, )
m.addLayerControl()
#m.addLayer(fields_boundary, {}, 'Fields')

palette = [
    '303000', '000000', '000000', '000000', '000000', 'f75cff', 
    '991695', 'e6ab2c', '3bef34', '2baa25', 'e6ef34', 'a1a524', 'eea15e'
]

palette_corrected = [
    '303000', '000000', '000000', '000000', '000000', 'f75cff', 
    '991695', 'e6ab2c', '3bef34', '2baa25', 'e6ef34', 'a1a524', 'eea15e', '29554E'
]

vis_params = {
    'min': 0,
    'max': 12,
    'palette': palette
}

vis_params_corrected = {
    'min': 0,
    'max': 13,
    'palette': palette_corrected
}

labels = """0. Background
1. Built-up
2. Water (Kharif)
3. Water (Kharif + Rabi)
4. Water (Kharif + Rabi + Zaid)
5. Croplands
6. Tree/Forest
7. Barren Land
8. Single Kharif (Light blue)
9. Single Non-Kharif
10. Double (Dark-Blue)
11. Triple
12. Shrub_Scrub
13. Plantation"""
labels = [i.split(". ")[-1] for i in labels.split("\n")]

palette_ = ['3bef34', '991695', '0000FF']
vis_params_ = {
    'min': 0,
    'max': 2,
    'palette': palette_
}
aez = ee.FeatureCollection("users/mtpictd/agro_eco_regions")

m.addLayer(mosaic_img.select("predicted_label"), vis_params_corrected, 'LULC_corrected')
m.addLayer(original_lulc_img, vis_params, 'LULC_original')
m.addLayer(aez, {}, "AEZ boundaries")
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

In [6]:
import re
original_lulc_img = ee.Image('projects/corestack-datasets/assets/datasets/LULC_v3_river_basin/pan_india_lulc_v3_2023_2024').select('predicted_label')
PARENT = 'projects/raman-461708/assets'

wasteland = ee.Image("projects/corestack-datasets/assets/datasets/WRI/wasteland_2015_2016_pan_india")

legend_class = {0:'Background',1:'Mining/ Industrial Wastelands', 2:'Scrub Land', 3:'Waterlogged Area', 4:'Degraded Land Under Plantation Crop', 5:'Sandy Area', 6:'Degraded Forest', 7:'Degraded Pastures/ Grazing Land',8:'Barren Rocky Area',8:'Gullied and Ravinous Land', 9:'Salt Affected Area', 11:'Shifting Cultivation', 12:'Snow Covered/ Glacial Area'}
wasteland_palette = ['#000000', '#4B4B4B','#9DB55A','#2B83BA','#A1DAB4','#EED8AE','#2E7D32','#D9EF8B','#C3804B','#D7301F','#F1B6DA','#8C6BB1','#E6F2FF'];
#Map.addLayer(wasteland_1516, {bands:['predicted_label'], min: 0, max: 12, palette: palette}, 'Wasteland_2015_2016');

# ---- List all image assets once (fast follow-up lookups) ----
def _basename(asset_name: str) -> str:
    return asset_name.split('/')[-1]

all_assets = ee.data.listAssets({'parent': PARENT}).get('assets', [])
image_assets = [a for a in all_assets if a.get('type','').lower() == 'image']
basenames = {_basename(a['name']) for a in image_assets}

def asset_exists_base(base: str) -> bool:
    return base in basenames

def fullpath(base: str) -> str:
    return f'{PARENT}/{base}'

def list_part_bases_for_aez(aez_no: int):
    """Return sorted basenames for all parts like AEZ_{n}_v4_local_partK (K numeric)."""
    rx = re.compile(rf'^AEZ_{aez_no}_v4_local_part(\d+)$')
    parts = []
    for b in basenames:
        m = rx.match(b)
        if m:
            parts.append((int(m.group(1)), b))
    # Sort by numeric suffix to keep deterministic order (not required for mosaic, but nice to have).
    parts.sort(key=lambda x: x[0])
    return [b for _, b in parts]

def choose_assets_for_aez(aez_no: int):
    """
    Preference:
      1) AEZ_{n}_v4
      2) AEZ_{n}_v4_local_part*
      3) AEZ_{n}_v4_local
    Returns: list of full asset IDs (possibly multiple if parts), plus a metadata dict.
    """
    base_v4 = f'AEZ_{aez_no}_v4'
    base_local = f'AEZ_{aez_no}_v4_local'
    part_bases = list_part_bases_for_aez(aez_no)

    if asset_exists_base(base_v4):
        return [fullpath(base_v4)], {'source':'v4','parts':[]}

    if part_bases:
        return [fullpath(b) for b in part_bases], {'source':'v4_local_parts','parts':part_bases}

    if asset_exists_base(base_local):
        return [fullpath(base_local)], {'source':'v4_local','parts':[]}

    return [], {'source':'missing','parts':[]}

# ---- Collect images AEZ 1..19 ----
images = []
report = {}   # aez_no -> {'source':..., 'parts':[...] }
missing = []

for aez_no in range(1, 20):
    asset_ids, meta = choose_assets_for_aez(aez_no)
    report[aez_no] = meta
    if asset_ids:
        # If this AEZ has multiple parts, add them all; mosaic will merge them.
        for aid in asset_ids:
            images.append(ee.Image(aid))
    else:
        missing.append(aez_no)

if not images:
    raise RuntimeError("No AEZ images found across v4 / v4_local_parts / v4_local")

# ---- Final mosaic across all gathered images ----
# Order: AEZ 1..19; within an AEZ, parts in ascending numeric order.
# ImageCollection.mosaic() draws later images on top of earlier ones.
mosaic_img = ee.ImageCollection(images).mosaic()
m = geemap.Map()
#m.centerObject(farm.limit(1))


url = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}'
m.layout.height = '1000px'
m.add_tile_layer(url, name="Google Map", attribution="Google")
#m.add_basemap(, )
m.addLayerControl()
#m.addLayer(fields_boundary, {}, 'Fields')

palette = [
    '303000', '000000', '000000', '000000', '000000', 'f75cff', 
    '991695', 'e6ab2c', '3bef34', '2baa25', 'e6ef34', 'a1a524', 'eea15e'
]

palette_corrected = [
    '303000', '000000', '000000', '000000', '000000', 'f75cff', 
    '991695', 'e6ab2c', '3bef34', '2baa25', 'e6ef34', 'a1a524', 'eea15e', '29554E'
]

vis_params = {
    'min': 0,
    'max': 12,
    'palette': palette
}

vis_params_corrected = {
    'min': 0,
    'max': 13,
    'palette': palette_corrected
}

labels = """0. Background
1. Built-up
2. Water (Kharif)
3. Water (Kharif + Rabi)
4. Water (Kharif + Rabi + Zaid)
5. Croplands
6. Tree/Forest
7. Barren Land
8. Single Kharif (Light blue)
9. Single Non-Kharif
10. Double (Dark-Blue)
11. Triple
12. Shrub_Scrub
13. Plantation"""
labels = [i.split(". ")[-1] for i in labels.split("\n")]

palette_ = ['3bef34', '991695', '0000FF']
vis_params_ = {
    'min': 0,
    'max': 2,
    'palette': palette_
}
aez = ee.FeatureCollection("users/mtpictd/agro_eco_regions")

m.addLayer(mosaic_img.select("predicted_label"), vis_params_corrected, 'LULC_corrected')
m.addLayer(original_lulc_img, vis_params, 'LULC_original')
m.addLayer(aez, {}, "AEZ boundaries")

mosaic = mosaic_img.select('predicted_label').toInt16()
wl = wasteland.select('b1').toInt16()

# --- Masks / conditions ---
# We only want to modify pixels where mosaic is 7 or 12
target_mask = mosaic.eq(7).Or(mosaic.eq(12))

# Consider WRI/wasteland only when it's non-background (non-zero) and not masked
wl_valid = wl.neq(0)  # True where class != 0

# Shift wasteland classes so they don't collide with 0..13:
# e.g., wl=1 -> 15, 2 -> 16, ... 12 -> 26. (wl=0 ignored via wl_valid)
wl_shifted = wl.updateMask(wl_valid).add(13).toInt16()

# Replacement only where BOTH: (mosaic in {7,12}) AND (wasteland valid)
replace_mask = target_mask.And(wl_valid)

# Apply replacement; elsewhere keep original mosaic
mosaic_patched = mosaic.where(replace_mask, wl_shifted)
                       
# --- Original LULC palette (0–13) ---
lulc_palette = [
    '303000',  # 0 Background
    '000000',  # 1 Built-up
    '000000',  # 2 Water (Kharif)
    '000000',  # 3 Water (Kharif + Rabi)
    '000000',  # 4 Water (Kharif + Rabi + Zaid)
    'f75cff',  # 5 Croplands
    '991695',  # 6 Tree/Forest
    'e6ab2c',  # 7 Barren Land
    '3bef34',  # 8 Single Kharif
    '2baa25',  # 9 Single Non-Kharif
    'e6ef34',  # 10 Double
    'a1a524',  # 11 Triple
    'eea15e',  # 12 Shrub_Scrub
    '29554E'   # 13 Plantation
]

# --- Wasteland palette (from class 1–12, shifted +14 → 15–26) ---
# The original palette provided:
wasteland_palette = [
 '#4B4B4B', '#2B83BA', '#A1DAB4', '#EED8AE',
    '#2E7D32', '#D9EF8B', '#C3804B', '#D7301F', '#F1B6DA', '#8C6BB1', '#E6F2FF'
]

# Shifted palette: prepend a dummy entry for 14 (so index aligns with class value)
# Class 14 will represent 'Background' or 'no wasteland'
shifted_wasteland_palette = wasteland_palette  # class 14→26

# --- Combined palette (0–26) ---
combined_palette = lulc_palette + shifted_wasteland_palette

# (Optional) Combined labels for reference
combined_labels = {
    0: 'Background',
    1: 'Built-up',
    2: 'Water (Kharif)',
    3: 'Water (Kharif + Rabi)',
    4: 'Water (Kharif + Rabi + Zaid)',
    5: 'Croplands',
    6: 'Tree/Forest',
    7: 'Barren Land',
    8: 'Single Kharif',
    9: 'Single Non-Kharif',
    10: 'Double',
    11: 'Triple',
    12: 'Shrub_Scrub',
    13: 'Plantation',
    14: 'Mining/Industrial Wastelands',
    15: 'Waterlogged Area',
    16: 'Degraded Land Under Plantation Crop',
    17: 'Sandy Area',
    18: 'Degraded Forest',
    19: 'Degraded Pastures/Grazing Land',
    20: 'Barren Rocky Area',
    21: 'Gullied and Ravinous Land',
    22: 'Salt Affected Area',
    23: 'Shifting Cultivation',
    24: 'Snow Covered/Glacial Area'
}

# --- Visualization example ---
# (in Earth Engine Python API)
vis_params_patched = {
    'min': 0,
    'max': len(combined_palette) - 1,
    'palette': combined_palette
}

def merge_and_shift_lulc(img: ee.Image) -> ee.Image:
    # Single-pass pixelwise transform:
    # if i == 15 -> 12
    # else if i > 15 -> i - 1
    # else keep i
    i = img.toInt16()
    out = i.expression(
        "i == 15 ? 12 : (i > 15 ? i - 1 : i)",
        {"i": i}
    ).rename(img.bandNames())
    return out.toInt16()

mosaic_patched = merge_and_shift_lulc(mosaic_patched)


                  
m.addLayer(mosaic_patched, vis_params_patched, 'LULC_corrected_patched')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

In [9]:
one_hit_fc = mosaic_patched.updateMask(mosaic_patched.eq(14)).sample(
        region=roi,
        scale=10,
        numPixels=1,
        seed=37,
        geometries=False
    )
ee.Number(one_hit_fc.size()).gt(0).getInfo()

0

In [1]:
import ee, geemap
ee.Initialize(project='raman-461708')
ee.Authenticate()

roi = ee.FeatureCollection("users/mtpictd/agro_eco_regions").filter(ee.Filter.eq("ae_regcode", 8))
lulc_v3_2024 = ee.Image('projects/corestack-datasets/assets/datasets/LULC_v3_river_basin/pan_india_lulc_v3_2023_2024').clip(roi).select('predicted_label')
lulc_v3_2023 = ee.Image('projects/corestack-datasets/assets/datasets/LULC_v3_river_basin/pan_india_lulc_v3_2022_2023').clip(roi).select('predicted_label')
lulc_v3_2022 = ee.Image('projects/corestack-datasets/assets/datasets/LULC_v3_river_basin/pan_india_lulc_v3_2021_2022').clip(roi).select('predicted_label')
lulc_v3_2021 = ee.Image('projects/corestack-datasets/assets/datasets/LULC_v3_river_basin/pan_india_lulc_v3_2020_2021').clip(roi).select('predicted_label')
lulc_v3_2020 = ee.Image('projects/corestack-datasets/assets/datasets/LULC_v3_river_basin/pan_india_lulc_v3_2019_2020').clip(roi).select('predicted_label')
lulc_v3_2019 = ee.Image('projects/corestack-datasets/assets/datasets/LULC_v3_river_basin/pan_india_lulc_v3_2018_2019').clip(roi).select('predicted_label')
lulc_v3_2018 = ee.Image('projects/corestack-datasets/assets/datasets/LULC_v3_river_basin/pan_india_lulc_v3_2017_2018').clip(roi).select('predicted_label')

lulc_v4_2024 = ee.Image('projects/raman-461708/assets/AEZ_8_v4_temporal__2023-07-01_2024-06-30').select('predicted_label')
lulc_v4_2023 = ee.Image('projects/raman-461708/assets/AEZ_8_v4_temporal__2022-07-01_2023-06-30').select('predicted_label')
lulc_v4_2022 = ee.Image('projects/raman-461708/assets/AEZ_8_v4_temporal__2021-07-01_2022-06-30').select('predicted_label')
lulc_v4_2021 = ee.Image('projects/raman-461708/assets/AEZ_8_v4_temporal__2020-07-01_2021-06-30').select('predicted_label')
lulc_v4_2020 = ee.Image('projects/raman-461708/assets/AEZ_8_v4_temporal__2019-07-01_2020-06-30').select('predicted_label')
lulc_v4_2019 = ee.Image('projects/raman-461708/assets/AEZ_8_v4_temporal__2018-07-01_2019-06-30').select('predicted_label')
lulc_v4_2018 = ee.Image('projects/raman-461708/assets/AEZ_8_v4_temporal__2017-07-01_2018-06-30').select('predicted_label')

lulc_v4_2024_3 = ee.Image('projects/raman-461708/assets/AEZ_8_v4_temporal_3years__2023-07-01_2024-06-30').select('predicted_label')
lulc_v4_2023_3 = ee.Image('projects/raman-461708/assets/AEZ_8_v4_temporal_3years__2022-07-01_2023-06-30').select('predicted_label')
lulc_v4_2022_3 = ee.Image('projects/raman-461708/assets/AEZ_8_v4_temporal_3years__2021-07-01_2022-06-30').select('predicted_label')
lulc_v4_2021_3 = ee.Image('projects/raman-461708/assets/AEZ_8_v4_temporal_3years__2020-07-01_2021-06-30').select('predicted_label')
lulc_v4_2020_3 = ee.Image('projects/raman-461708/assets/AEZ_8_v4_temporal_3years__2019-07-01_2020-06-30').select('predicted_label')
lulc_v4_2019_3 = ee.Image('projects/raman-461708/assets/AEZ_8_v4_temporal_3years__2018-07-01_2019-06-30').select('predicted_label')
lulc_v4_2018_3 = ee.Image('projects/raman-461708/assets/AEZ_8_v4_temporal_3years__2017-07-01_2018-06-30').select('predicted_label')

m = geemap.Map()
#m.centerObject(farm.limit(1))


url = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}'
m.layout.height = '1000px'
m.add_tile_layer(url, name="Google Map", attribution="Google")
#m.add_basemap(, )
m.addLayerControl()
#m.addLayer(fields_boundary, {}, 'Fields')

palette = [
    '303000', '000000', '000000', '000000', '000000', 'f75cff', 
    '991695', 'e6ab2c', '3bef34', '2baa25', 'e6ef34', 'a1a524', 'eea15e'
]

palette_corrected = [
    '303000', '000000', '000000', '000000', '000000', 'f75cff', 
    '991695', 'e6ab2c', '3bef34', '2baa25', 'e6ef34', 'a1a524', 'eea15e', '29554E'
]

vis_params = {
    'min': 0,
    'max': 12,
    'palette': palette
}

vis_params_corrected = {
    'min': 0,
    'max': 13,
    'palette': palette_corrected
}

labels = """0. Background
1. Built-up
2. Water (Kharif)
3. Water (Kharif + Rabi)
4. Water (Kharif + Rabi + Zaid)
5. Croplands
6. Tree/Forest
7. Barren Land
8. Single Kharif (Light blue)
9. Single Non-Kharif
10. Double (Dark-Blue)
11. Triple
12. Shrub_Scrub
13. Plantation"""
labels = [i.split(". ")[-1] for i in labels.split("\n")]

palette_ = ['3bef34', '991695', '0000FF']
vis_params_ = {
    'min': 0,
    'max': 2,
    'palette': palette_
}

m.addLayer(lulc_v4_2024, vis_params_corrected, 'LULC_v4_2024')
m.addLayer(lulc_v4_2023, vis_params_corrected, 'LULC_v4_2023')
m.addLayer(lulc_v4_2022, vis_params_corrected, 'LULC_v4_2022')
m.addLayer(lulc_v4_2021, vis_params_corrected, 'LULC_v4_2021')
m.addLayer(lulc_v4_2020, vis_params_corrected, 'LULC_v4_2020')
m.addLayer(lulc_v4_2019, vis_params_corrected, 'LULC_v4_2019')
m.addLayer(lulc_v4_2018, vis_params_corrected, 'LULC_v4_2018')

m.addLayer(lulc_v4_2024_3, vis_params_corrected, 'LULC_v4_2024_3')
m.addLayer(lulc_v4_2023_3, vis_params_corrected, 'LULC_v4_2023_3')
m.addLayer(lulc_v4_2022_3, vis_params_corrected, 'LULC_v4_2022_3')
m.addLayer(lulc_v4_2021_3, vis_params_corrected, 'LULC_v4_2021_3')
m.addLayer(lulc_v4_2020_3, vis_params_corrected, 'LULC_v4_2020_3')
m.addLayer(lulc_v4_2019_3, vis_params_corrected, 'LULC_v4_2019_3')
m.addLayer(lulc_v4_2018_3, vis_params_corrected, 'LULC_v4_2018_3')


m.addLayer(lulc_v3_2024, vis_params, 'LULC_v3_2024')
m.addLayer(lulc_v3_2023, vis_params, 'LULC_v3_2023')
m.addLayer(lulc_v3_2022, vis_params, 'LULC_v3_2022')
m.addLayer(lulc_v3_2021, vis_params, 'LULC_v3_2021')
m.addLayer(lulc_v3_2020, vis_params, 'LULC_v3_2020')
m.addLayer(lulc_v3_2019, vis_params, 'LULC_v3_2019')
m.addLayer(lulc_v3_2018, vis_params, 'LULC_v3_2018')


m

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

In [ ]:
import ee, geemap
ee.Initialize(project='raman-461708')
ee.Authenticate()

roi = ee.FeatureCollection("users/mtpictd/agro_eco_regions").filter(ee.Filter.eq("ae_regcode", 8))
lulc_v3_2024 = ee.Image('projects/corestack-datasets/assets/datasets/LULC_v3_river_basin/pan_india_lulc_v3_2023_2024').clip(roi).select('predicted_label')
lulc_v3_2023 = ee.Image('projects/corestack-datasets/assets/datasets/LULC_v3_river_basin/pan_india_lulc_v3_2022_2023').clip(roi).select('predicted_label')
lulc_v3_2022 = ee.Image('projects/corestack-datasets/assets/datasets/LULC_v3_river_basin/pan_india_lulc_v3_2021_2022').clip(roi).select('predicted_label')
lulc_v3_2021 = ee.Image('projects/corestack-datasets/assets/datasets/LULC_v3_river_basin/pan_india_lulc_v3_2020_2021').clip(roi).select('predicted_label')
lulc_v3_2020 = ee.Image('projects/corestack-datasets/assets/datasets/LULC_v3_river_basin/pan_india_lulc_v3_2019_2020').clip(roi).select('predicted_label')
lulc_v3_2019 = ee.Image('projects/corestack-datasets/assets/datasets/LULC_v3_river_basin/pan_india_lulc_v3_2018_2019').clip(roi).select('predicted_label')
lulc_v3_2018 = ee.Image('projects/corestack-datasets/assets/datasets/LULC_v3_river_basin/pan_india_lulc_v3_2017_2018').clip(roi).select('predicted_label')

lulc_v4_2024_3 = ee.Image('projects/raman-461708/assets/AEZ_8_v4_temporal_3years__2023-07-01_2024-06-30').select('predicted_label')
lulc_v4_2023_3 = ee.Image('projects/raman-461708/assets/AEZ_8_v4_temporal_3years__2022-07-01_2023-06-30').select('predicted_label')
lulc_v4_2022_3 = ee.Image('projects/raman-461708/assets/AEZ_8_v4_temporal_3years__2021-07-01_2022-06-30').select('predicted_label')
lulc_v4_2021_3 = ee.Image('projects/raman-461708/assets/AEZ_8_v4_temporal_3years__2020-07-01_2021-06-30').select('predicted_label')
lulc_v4_2020_3 = ee.Image('projects/raman-461708/assets/AEZ_8_v4_temporal_3years__2019-07-01_2020-06-30').select('predicted_label')
lulc_v4_2019_3 = ee.Image('projects/raman-461708/assets/AEZ_8_v4_temporal_3years__2018-07-01_2019-06-30').select('predicted_label')
lulc_v4_2018_3 = ee.Image('projects/raman-461708/assets/AEZ_8_v4_temporal_3years__2017-07-01_2018-06-30').select('predicted_label')

m = geemap.Map()
#m.centerObject(farm.limit(1))


url = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}'
m.layout.height = '1000px'
m.add_tile_layer(url, name="Google Map", attribution="Google")
#m.add_basemap(, )
m.addLayerControl()
#m.addLayer(fields_boundary, {}, 'Fields')

palette = [
    '303000', '000000', '000000', '000000', '000000', 'f75cff', 
    '991695', 'e6ab2c', '3bef34', '2baa25', 'e6ef34', 'a1a524', 'eea15e'
]

palette_corrected = [
    '303000', '000000', '000000', '000000', '000000', 'f75cff', 
    '991695', 'e6ab2c', '3bef34', '2baa25', 'e6ef34', 'a1a524', 'eea15e', '29554E'
]

vis_params = {
    'min': 0,
    'max': 12,
    'palette': palette
}

vis_params_corrected = {
    'min': 0,
    'max': 13,
    'palette': palette_corrected
}

labels = """0. Background
1. Built-up
2. Water (Kharif)
3. Water (Kharif + Rabi)
4. Water (Kharif + Rabi + Zaid)
5. Croplands
6. Tree/Forest
7. Barren Land
8. Single Kharif (Light blue)
9. Single Non-Kharif
10. Double (Dark-Blue)
11. Triple
12. Shrub_Scrub
13. Plantation"""
labels = [i.split(". ")[-1] for i in labels.split("\n")]

palette_ = ['3bef34', '991695', '0000FF']
vis_params_ = {
    'min': 0,
    'max': 2,
    'palette': palette_
}

m.addLayer(lulc_v4_2024, vis_params_corrected, 'LULC_v4_2024')
m.addLayer(lulc_v4_2023, vis_params_corrected, 'LULC_v4_2023')
m.addLayer(lulc_v4_2022, vis_params_corrected, 'LULC_v4_2022')
m.addLayer(lulc_v4_2021, vis_params_corrected, 'LULC_v4_2021')
m.addLayer(lulc_v4_2020, vis_params_corrected, 'LULC_v4_2020')
m.addLayer(lulc_v4_2019, vis_params_corrected, 'LULC_v4_2019')
m.addLayer(lulc_v4_2018, vis_params_corrected, 'LULC_v4_2018')
m.addLayer(lulc_v3_2024, vis_params, 'LULC_v3_2024')
m.addLayer(lulc_v3_2023, vis_params, 'LULC_v3_2023')
m.addLayer(lulc_v3_2022, vis_params, 'LULC_v3_2022')
m.addLayer(lulc_v3_2021, vis_params, 'LULC_v3_2021')
m.addLayer(lulc_v3_2020, vis_params, 'LULC_v3_2020')
m.addLayer(lulc_v3_2019, vis_params, 'LULC_v3_2019')
m.addLayer(lulc_v3_2018, vis_params, 'LULC_v3_2018')


m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…